## **Rede Neural - Classificação** <br> COC361 - Inteligência Computacional (2021.2)
### Alunos: <br> Henrique Chaves (DRE 119025571) <br> Pedro Boechat (DRE 119065050)
<hr>

### • Importação das bibliotecas

In [1]:
# Bibliotecas padrão
from os import (
    listdir,
    makedirs
)

# Bibliotecas para manipulação dos dados
import kaggle
import numpy as np
import pandas as pd

# SKLearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay
)

# Tensorflow/Keras
from tensorflow.keras.layers import (
    Dense,
    Dropout
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    ReduceLROnPlateau
)
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.config import list_physical_devices

# Bibliotecas para plot
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Carregamento das variáveis de ambiente
from dotenv import load_dotenv
load_dotenv()

# Número de GPUs disponíveis para o Tensorflow/Keras
print("Número de GPUs disponíveis: ", len(list_physical_devices('GPU')))

Número de GPUs disponíveis:  1


### • Download do dataset ([Link](https://www.kaggle.com/mlg-ulb/creditcardfraud?select=creditcard.csv))

In [5]:
# Cria pasta de destino, caso não exista
makedirs("./data/classification", exist_ok=True)

# Se a pasta de destino estiver vazia, baixa os dados
if len(listdir("./data/classification/")) == 0:
    kaggle.api.dataset_download_file(
        "mathchi/diabetes-data-set",
        "diabetes.csv",
        "./data/classification/"
    )

### • Carregamento do dataset

In [6]:
# Carregamento dos dados
df = pd.read_csv("./data/classification/diabetes.csv")

### • Divisão do dataset em treino e teste

In [8]:
# Divisão do dataset em treino e teste, com a coluna `Class` como y
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:,:-1],
    df.iloc[:,-1],
    test_size=0.2,
    random_state=42
)

### • Definição de callbacks da rede

In [ ]:
# Salva o modelo a cada melhora
checkpoint = ModelCheckpoint(
    "./data/classification/checkpoints",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

# Reduz a learning rate caso o modelo esteja estagnado
lr_reduce = ReduceLROnPlateau(
    monitor="val_accuracy",
    factor=0.1,
    min_delta=1e-5,
    patience=5,
    verbose=1
)

# Lista contendo os checkpoints definidos
callbacks = [checkpoint, lr_reduce]

### • Definição das camadas da rede

In [ ]:
# Criação do modelo sequencial
model = models.Sequential()

# Camada de pooling
model.add(layers.GlobalAveragePooling2D())

# Camada de normalização
model.add(layers.BatchNormalization())

# Camada de achatamento
model.add(layers.Flatten())

# Camada de adensamento com ativação RELU
model.add(layers.Dense(128, activation='relu'))

# Camada de dropout
model.add(layers.Dropout(0.2))

# Camada de adensamento com ativação SOFTMAX
model.add(layers.Dense(3, activation='softmax'))

### • Compilação da rede

In [ ]:
# Compilação do modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### • Treino da rede

In [ ]:
# Treino do modelo
history = model.fit(
    X_train,
    validation_data=(
        X_test,
        y_test
    ),
    callbacks=callbacks,
    epochs=100,
    batch_size=32
)

### • Avaliação da rede

In [ ]:
# Definição dos subplots
fig, ax = plt.subplots(figsize=(15, 16), nrows=2)

# Gráfico da acurácia do modelo por época
ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Acurácia do modelo por época', fontsize=18)
ax[0].set_ylabel('Acurácia', fontsize=14)
ax[0].set_xlabel('Época', fontsize=14)
ax[0].legend(['Treino', 'Teste'], loc='upper left', fontsize=16)

# Gráfico da loss do modelo por época
ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[0].set_title('Loss do modelo por época', fontsize=18)
ax[0].set_ylabel('Loss', fontsize=14)
ax[0].set_xlabel('Época', fontsize=14)
ax[0].legend(['Treino', 'Teste'], loc='upper left', fontsize=16)

# Ajuste do layout do plot
plt.tight_layout()

### • Predição usando a rede treinada

In [ ]:
y_test_pred = np.argmax(model.predict(X_test), axis=1)

### • Matriz de confusão

In [ ]:
# Criação da matriz de confusão
cm = confusion_matrix(np.argmax(y_test, axis=1), y_test_pred)

# Criação da visualização da matriz de confusão
cm_plt = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=[0,1]
)

# Exibição da matriz de confusão
cm_plt.plot(cmap=plt.cm.Blues)
plt.show()